In [41]:
'''
Analysis on how the ownership of a token holder change
'''

'\nAnalysis on how the ownership of a token holder change\n'

In [42]:
from bs4 import BeautifulSoup
import urllib3
import re

In [54]:
baseUrl='https://etherscan.io/'
funUrl_chart='token/tokenholderchart/'
tokenid='0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0'

pool=urllib3.PoolManager()
r=pool.request('GET',baseUrl+funUrl_chart+tokenid)
webdata=r.data

soup = BeautifulSoup(webdata)
table = soup.find("table", attrs={"class":"table table-hover "})
headings = [th.get_text() for th in table.find("tr").find_all("th")]
datasets = []
for row in table.find_all("tr")[1:]:
    dataset = zip(headings, (td.get_text() for td in row.find_all("td")))
    datasets.append(dataset)

'''
Clean the data
'''
for idx in range(len(datasets)):
    for idxx in range(len(datasets[idx])):
        tup=[]
        for idxxx in range(len(datasets[idx][idxx])):
            tmp=re.sub(r'\([^)]*\)', '', datasets[idx][idxx][idxxx])
            tmp=tmp.strip()
            tmp=str(tmp)
            tup.append(tmp)
        tupp=tuple(tup)
        datasets[idx][idxx]=tup

datasets

/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[[['Rank', '1'],
  ['Address', '0xd0a6e6c54dbc68db5db3a091b171a77407ff7ccf'],
  ['Quantity', '661135124.032559140831597903'],
  ['Percentage', '66.1135%']],
 [['Rank', '2'],
  ['Address', '0x00000000000000000000000000000000000000b1'],
  ['Quantity', '100000000'],
  ['Percentage', '10.0000%']],
 [['Rank', '3'],
  ['Address', '0x42da8a05cb7ed9a43572b5ba1b8f82a0a6e263dc'],
  ['Quantity', '30160347.008970204988273462'],
  ['Percentage', '3.0160%']],
 [['Rank', '4'],
  ['Address', '0xf4b51b14b9ee30dc37ec970b50a486f37686e2a8'],
  ['Quantity', '14386105.99177623992561125'],
  ['Percentage', '1.4386%']],
 [['Rank', '5'],
  ['Address', '0x174443351e21d47ed9ab51517a301107d92ede64'],
  ['Quantity', '13896395.883908884146577035'],
  ['Percentage', '1.3896%']],
 [['Rank', '6'],
  ['Address', '0xe2dd88165572932944ba72ab6ed143f9a9855a81'],
  ['Quantity', '4400000'],
  ['Percentage', '0.4400%']],
 [['Rank', '7'],
  ['Address', '0xb819dfe8927f1055467c957083a752e2f67e4d65'],
  ['Quantity', '3501904.1259